# PEFT fine-tuning

En este notebook, ajustaremos un LLM existente de Hugging Face para mejorar el resumen de diálogos.

Modelo Base: [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5)

## Setup

In [ ]:
!pip install -qU torch==2.6.0 torchvision==0.21.0+cu124 torchaudio==2.6.0+cu124 --index-url https://download.pytorch.org/whl/cu124

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 15.3 MB/s eta 0:00:00


In [ ]:
!pip install -qU torchdata py7zr transformers datasets evaluate rouge_score loralib peft

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
from random import randrange

Dataset [Samsum](https://huggingface.co/datasets/samsum)

In [ ]:
dataset = load_dataset("Samsung/samsum")

The repository for Samsung/samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Samsung/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
def print_text(title, dialogue, summary, model_summary):
    print(dash_line)
    print(title)
    print(dash_line)
    print('Dialogue')
    print(dash_line)
    print(dialogue)
    print()
    print(dash_line)
    print('Summary')
    print(dash_line)
    print(summary)
    print()
    print(dash_line)
    print('Model Summary')
    print(dash_line)
    print(model_summary)
    print(dash_line)
    print()
    print()

dash_line = '*' * 100

In [ ]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Función que extrae el número de parámetros entrenables

In [ ]:
def model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"Parametros entrenables: {trainable_model_params}\nTodos los parametros: {all_model_params}\Porcentaje: {100 * trainable_model_params / all_model_params:.2f}%"

print(model_parameters(original_model))

Parametros entrenables: 247577856
Todos los parametros: 247577856\Porcentaje: 100.00%


## Test con Zero Shot Inferencing


In [ ]:
index = randrange(len(dataset["test"]))

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

print_text(f'Example {index}', dialogue, summary, output)

****************************************************************************************************
Example 138
****************************************************************************************************
Dialogue
****************************************************************************************************
Willyx: how did your job interview go?
Alicja: it's tomorrow :D
Willyx: sorry
Alicja: it's ok :)
Willyx: let me know how it went
Alicja: sure

****************************************************************************************************
Summary
****************************************************************************************************
Alicja's job interview is tomorrow. She will inform Willyx how it goes.

****************************************************************************************************
Model Summary
****************************************************************************************************
Alicja's job interview is tomorrow

## Fine-tuning

### Preprocess Dataset

The function takes a batch of examples containing dialogues and their corresponding summaries. For each dialogue, it adds a specific format indicating to the model the task of summarizing the dialogue. Subsequently, it tokenizes both the formatted dialogues and the summaries.

```
Summarize the following dialogue.

    Darth Vader: Luke, yo soy tu padre.
    Luke Skywalker: Nooooooooooooooooooooooooo
    
Summary:
```

Training response (summary):
```
Darth Vader tells Luke Skywalker that he is his father
```

In [1]:
def tokenize_function(example):
    start_prompt = 'Summarize the following dialogue.\n\n'
    end_prompt = '\n\nSummary: '

    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]

    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids

    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(['id', 'dialogue', 'summary',])


NameError: name 'dataset' is not defined

Usaremos un subconjunto del conjunto de datos para no pasar tanto tiempo entrenando.



In [ ]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Filter:   0%|          | 0/14732 [00:00<?, ? examples/s]

Filter:   0%|          | 0/819 [00:00<?, ? examples/s]

Filter:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Training: (148, 2)
Validation: (9, 2)
Test: (9, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 148
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 9
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 9
    })
})


### Fine-Tuning

Configuraremos el `Trainer` de HuggingFace => [Documentación](https://huggingface.co/docs/transformers/main_classes/trainer).









In [ ]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=10,
    logging_steps=50,
    hub_model_id="ericrisco/flan-t5-base-samsum"
    #hub_strategy="every_save"
    #max_steps=10
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: erisco (keepcoding) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
50,45.095000
100,44.105000
150,43.610000


TrainOutput(global_step=190, training_loss=44.11184210526316, metrics={'train_runtime': 42.2935, 'train_samples_per_second': 34.994, 'train_steps_per_second': 4.492, 'total_flos': 1013440881623040.0, 'train_loss': 44.11184210526316, 'epoch': 10.0})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1746591034.ca3888720d1f.2460.0:   0%|          | 0.00/6.93k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ericrisco/flan-t5-base-samsum/commit/a71cebd062fc4bffbc3da4ad3b06c201cf87282a', commit_message='End of training', commit_description='', oid='a71cebd062fc4bffbc3da4ad3b06c201cf87282a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ericrisco/flan-t5-base-samsum', endpoint='https://huggingface.co', repo_type='model', repo_id='ericrisco/flan-t5-base-samsum'), pr_revision=None, pr_num=None)

## Model Evaluation


Hay una métrica para evaluar si un resumen está bien hecho. Esta evaluación es la métrica ROUGE. Podemos aplicar esta métrica al modelo que hemos entrenado o podemos tomar otro modelo de HF si habéis detenido el entrenamiento.

In [ ]:
#instruct_model_name='ybagoury/flan-t5-base-tldr_news'
instruct_model_name='ericrisco/flan-t5-base-samsum'

instruct_model = AutoModelForSeq2SeqLM.from_pretrained(instruct_model_name)

rouge = evaluate.load('rouge')

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

Compararemos el resultado de 5 resumenes de un dialogo


In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
original_model.to(device)
instruct_model.to(device)

dialogues = dataset['test'][0:5]['dialogue']
human_baseline_summaries = dataset['test'][0:5]['summary']

original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    original_model_outputs = original_model.generate(input_ids=input_ids, max_new_tokens=200)
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, max_new_tokens=200)
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)

zipped_summaries = list(zip(dialogues, human_baseline_summaries, original_model_summaries, instruct_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['dialogues', 'human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])


for i, row in df.iterrows():
    print(f"Resumen #{i+1}")
    print("Dialogo:")
    print(row['dialogues'])
    print("\n" + "-"*80 + "\n")
    print("Resumen humano:")
    print(row['human_baseline_summaries'])
    print("\nResumen modelo:")
    print(row['original_model_summaries'])
    print("\nResumen finetuning")
    print(row['instruct_model_summaries'])
    print("\n" + "-"*80 + "\n")
    print("\n" + "-"*80 + "\n")

Resumen #1
Dialogo:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

--------------------------------------------------------------------------------

Resumen humano:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Resumen modelo:
Hannah and Amanda want to know Betty's number.

Resumen finetuning
Amanda can't find Betty's number. Amanda will ask Larry.

--------------------------------------------------------------------------------


--------------------------------------------------------------------------------

Resumen #2
Dialogo:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how

In [ ]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': np.float64(0.4450672268907564), 'rouge2': np.float64(0.19860972360972362), 'rougeL': np.float64(0.3738403361344538), 'rougeLsum': np.float64(0.37242857142857144)}
INSTRUCT MODEL:
{'rouge1': np.float64(0.5182208542331661), 'rouge2': np.float64(0.2677848056949469), 'rougeL': np.float64(0.45299310938845816), 'rougeLsum': np.float64(0.44950727060850176)}
